# Treinamento Normal Sem PSO

## AlexNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
import random

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função para criar um subconjunto dos dados
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)


# Função para selecionar o modelo sem pré-treinamento
def select_model(architecture):
    if architecture == 'alexnet':
        model = models.alexnet(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'vgg11':
        model = models.vgg11(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'resnet18':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 10)
    else:
        raise ValueError("Arquitetura não suportada: escolha 'alexnet', 'vgg11' ou 'resnet18'")
    return model.to(device)

# Transformações de dados com normalizações para modelos pré-treinados
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Reduzir a quantidade de dados usados
train_subset_size = 50000  # Número ainda mais reduzido de exemplos de treino
test_subset_size = 10000    # Número ainda mais reduzido de exemplos de teste

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_subset = create_subset(trainset, train_subset_size)
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)  # Reduzir o batch_size

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_subset = create_subset(testset, test_subset_size)
testloader = DataLoader(test_subset, batch_size=64, shuffle=False, num_workers=2)  # Reduzir o batch_size

# Carregar o conjunto de dados CIFAR-10
#trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Criar DataLoader
#trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Função para treinamento
def train_model(model, criterion, optimizer, num_epochs=10):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        val_loss, val_accuracy = evaluate_model(model, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    print('Treinamento concluído.')
    return model

# Função para avaliação
def evaluate_model(model, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(testloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Definir arquitetura e otimizador
architecture = 'alexnet'  # Escolha entre 'alexnet', 'vgg11' ou 'resnet18'
optimizer_choice = 'Adam'  # Escolha entre 'AdaGrad', 'RMSProp' ou 'Adam'

model = select_model(architecture)
criterion = nn.CrossEntropyLoss()

# Selecionar o otimizador
if optimizer_choice == 'AdaGrad':
    optimizer = optim.Adagrad(model.parameters(), lr=0.01)
elif optimizer_choice == 'RMSProp':
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
elif optimizer_choice == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
else:
    raise ValueError("Otimizador não suportado: escolha 'AdaGrad', 'RMSProp' ou 'Adam'")

# Treinar o modelo
num_epochs = 30
trained_model = train_model(model, criterion, optimizer, num_epochs)

# Avaliação final
final_loss, final_accuracy = evaluate_model(trained_model, criterion)
print(f'Avaliação final - Loss: {final_loss:.4f}, Accuracy: {final_accuracy:.4f}')

# Salvando o modelo treinado
torch.save(trained_model.state_dict(), f'trained_model_{architecture}_{optimizer_choice}.pth')


## VGG11

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
import random

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função para criar um subconjunto dos dados
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)


# Função para selecionar o modelo sem pré-treinamento
def select_model(architecture):
    if architecture == 'alexnet':
        model = models.alexnet(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'vgg11':
        model = models.vgg11(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'resnet18':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 10)
    else:
        raise ValueError("Arquitetura não suportada: escolha 'alexnet', 'vgg11' ou 'resnet18'")
    return model.to(device)

# Transformações de dados com normalizações para modelos pré-treinados
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Reduzir a quantidade de dados usados
train_subset_size = 50000  # Número ainda mais reduzido de exemplos de treino
test_subset_size = 10000    # Número ainda mais reduzido de exemplos de teste

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_subset = create_subset(trainset, train_subset_size)
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)  # Reduzir o batch_size

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_subset = create_subset(testset, test_subset_size)
testloader = DataLoader(test_subset, batch_size=64, shuffle=False, num_workers=2)  # Reduzir o batch_size

# Carregar o conjunto de dados CIFAR-10
#trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Criar DataLoader
#trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Função para treinamento
def train_model(model, criterion, optimizer, num_epochs=10):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        val_loss, val_accuracy = evaluate_model(model, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    print('Treinamento concluído.')
    return model

# Função para avaliação
def evaluate_model(model, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(testloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Definir arquitetura e otimizador
architecture = 'vgg11'  # Escolha entre 'alexnet', 'vgg11' ou 'resnet18'
optimizer_choice = 'Adam'  # Escolha entre 'AdaGrad', 'RMSProp' ou 'Adam'

model = select_model(architecture)
criterion = nn.CrossEntropyLoss()

# Selecionar o otimizador
if optimizer_choice == 'AdaGrad':
    optimizer = optim.Adagrad(model.parameters(), lr=0.01)
elif optimizer_choice == 'RMSProp':
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
elif optimizer_choice == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
else:
    raise ValueError("Otimizador não suportado: escolha 'AdaGrad', 'RMSProp' ou 'Adam'")

# Treinar o modelo
num_epochs = 30
trained_model = train_model(model, criterion, optimizer, num_epochs)

# Avaliação final
final_loss, final_accuracy = evaluate_model(trained_model, criterion)
print(f'Avaliação final - Loss: {final_loss:.4f}, Accuracy: {final_accuracy:.4f}')

# Salvando o modelo treinado
torch.save(trained_model.state_dict(), f'trained_model_{architecture}_{optimizer_choice}.pth')

## ResNet18

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
import random

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função para criar um subconjunto dos dados
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)


# Função para selecionar o modelo sem pré-treinamento
def select_model(architecture):
    if architecture == 'alexnet':
        model = models.alexnet(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'vgg11':
        model = models.vgg11(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'resnet18':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 10)
    else:
        raise ValueError("Arquitetura não suportada: escolha 'alexnet', 'vgg11' ou 'resnet18'")
    return model.to(device)


# Transformações de dados com normalizações para modelos pré-treinados
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Reduzir a quantidade de dados usados
train_subset_size = 50000  # Número ainda mais reduzido de exemplos de treino
test_subset_size =10000    # Número ainda mais reduzido de exemplos de teste

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_subset = create_subset(trainset, train_subset_size)
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)  # Reduzir o batch_size

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_subset = create_subset(testset, test_subset_size)
testloader = DataLoader(test_subset, batch_size=64, shuffle=False, num_workers=2)  # Reduzir o batch_size

# Carregar o conjunto de dados CIFAR-10
#trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Criar DataLoader
#trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Função para treinamento
def train_model(model, criterion, optimizer, num_epochs=10):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        val_loss, val_accuracy = evaluate_model(model, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    print('Treinamento concluído.')
    return model

# Função para avaliação
def evaluate_model(model, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(testloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Definir arquitetura e otimizador
architecture = 'resnet18'  # Escolha entre 'alexnet', 'vgg11' ou 'resnet18'
optimizer_choice = 'Adam'  # Escolha entre 'AdaGrad', 'RMSProp' ou 'Adam'

model = select_model(architecture)
criterion = nn.CrossEntropyLoss()

# Selecionar o otimizador
if optimizer_choice == 'AdaGrad':
    optimizer = optim.Adagrad(model.parameters(), lr=0.01)
elif optimizer_choice == 'RMSProp':
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
elif optimizer_choice == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
else:
    raise ValueError("Otimizador não suportado: escolha 'AdaGrad', 'RMSProp' ou 'Adam'")

# Treinar o modelo
num_epochs = 30
trained_model = train_model(model, criterion, optimizer, num_epochs)

# Avaliação final
final_loss, final_accuracy = evaluate_model(trained_model, criterion)
print(f'Avaliação final - Loss: {final_loss:.4f}, Accuracy: {final_accuracy:.4f}')

# Salvando o modelo treinado
torch.save(trained_model.state_dict(), f'trained_model_{architecture}_{optimizer_choice}.pth')

## MobileNet_V2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
import random

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função para criar um subconjunto dos dados
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)


# Função para selecionar o modelo sem pré-treinamento
def select_model(architecture):
    if architecture == 'alexnet':
        model = models.alexnet(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'vgg11':
        model = models.vgg11(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'resnet18':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif architecture == 'mobilenet_v2':
        model = models.mobilenet_v2(weights=None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    else:
        raise ValueError("Arquitetura não suportada: escolha 'alexnet', 'vgg11', 'resnet18' ou 'mobilenet_v2'")
    return model.to(device)


# Transformações de dados com normalizações para modelos pré-treinados
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Reduzir a quantidade de dados usados
train_subset_size = 50000  # Número ainda mais reduzido de exemplos de treino
test_subset_size = 10000    # Número ainda mais reduzido de exemplos de teste

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_subset = create_subset(trainset, train_subset_size)
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)  # Reduzir o batch_size

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_subset = create_subset(testset, test_subset_size)
testloader = DataLoader(test_subset, batch_size=64, shuffle=False, num_workers=2)  # Reduzir o batch_size

# Carregar o conjunto de dados CIFAR-10
#trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Criar DataLoader
#trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Função para treinamento
def train_model(model, criterion, optimizer, num_epochs=10):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        val_loss, val_accuracy = evaluate_model(model, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    print('Treinamento concluído.')
    return model

# Função para avaliação
def evaluate_model(model, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(testloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Definir arquitetura e otimizador
architecture = 'mobilenet_v2'  # Pode ser 'alexnet', 'vgg11', 'resnet18' ou 'mobilenet_v2'
optimizer_choice = 'Adam'  # Escolha entre 'AdaGrad', 'RMSProp' ou 'Adam'

model = select_model(architecture)
criterion = nn.CrossEntropyLoss()

# Selecionar o otimizador
if optimizer_choice == 'AdaGrad':
    optimizer = optim.Adagrad(model.parameters(), lr=0.01)
elif optimizer_choice == 'RMSProp':
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
elif optimizer_choice == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
else:
    raise ValueError("Otimizador não suportado: escolha 'AdaGrad', 'RMSProp' ou 'Adam'")

# Treinar o modelo
num_epochs = 30
trained_model = train_model(model, criterion, optimizer, num_epochs)

# Avaliação final
final_loss, final_accuracy = evaluate_model(trained_model, criterion)
print(f'Avaliação final - Loss: {final_loss:.4f}, Accuracy: {final_accuracy:.4f}')

# Salvando o modelo treinado
torch.save(trained_model.state_dict(), f'trained_model_{architecture}_{optimizer_choice}.pth')

## SqueezeNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
import random

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função para criar um subconjunto dos dados
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)


# Função para selecionar o modelo sem pré-treinamento
def select_model(architecture):
    if architecture == 'alexnet':
        model = models.alexnet(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'vgg11':
        model = models.vgg11(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'resnet18':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif architecture == 'squeezenet':
        model = models.squeezenet1_1(weights=None)
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
        model.num_classes = 10
    else:
        raise ValueError("Arquitetura não suportada: escolha 'alexnet', 'vgg11', 'resnet18' ou 'squeezenet'")

    return model.to(device)

# Transformações de dados com normalizações para modelos pré-treinados
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Reduzir a quantidade de dados usados
train_subset_size = 50000  # Número ainda mais reduzido de exemplos de treino
test_subset_size = 10000    # Número ainda mais reduzido de exemplos de teste

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_subset = create_subset(trainset, train_subset_size)
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)  # Reduzir o batch_size

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_subset = create_subset(testset, test_subset_size)
testloader = DataLoader(test_subset, batch_size=64, shuffle=False, num_workers=2)  # Reduzir o batch_size

# Carregar o conjunto de dados CIFAR-10
#trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Criar DataLoader
#trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Função para treinamento
def train_model(model, criterion, optimizer, num_epochs=10):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        val_loss, val_accuracy = evaluate_model(model, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    print('Treinamento concluído.')
    return model

# Função para avaliação
def evaluate_model(model, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(testloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Definir arquitetura e otimizador
architecture = 'squeezenet'  # Escolha entre 'alexnet', 'vgg11', 'resnet18' ou 'squeezenet'"
optimizer_choice = 'Adam'  # Escolha entre 'AdaGrad', 'RMSProp' ou 'Adam'

model = select_model(architecture)
criterion = nn.CrossEntropyLoss()

# Selecionar o otimizador
if optimizer_choice == 'AdaGrad':
    optimizer = optim.Adagrad(model.parameters(), lr=0.01)
elif optimizer_choice == 'RMSProp':
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
elif optimizer_choice == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
else:
    raise ValueError("Otimizador não suportado: escolha 'AdaGrad', 'RMSProp' ou 'Adam'")

# Treinar o modelo
num_epochs = 30
trained_model = train_model(model, criterion, optimizer, num_epochs)

# Avaliação final
final_loss, final_accuracy = evaluate_model(trained_model, criterion)
print(f'Avaliação final - Loss: {final_loss:.4f}, Accuracy: {final_accuracy:.4f}')

# Salvando o modelo treinado
torch.save(trained_model.state_dict(), f'trained_model_{architecture}_{optimizer_choice}.pth')